In [51]:
import numpy as np
from enum import Enum
from typing import List
import random
import math

# Things to do
- Initialize n agents, n/2 on each team (team A, B)
- start with 1 topic for opinions
- give uniform/normal dist of [-1, 0), (0, -1] for team A, B respectively
- set some tolerance threshold for if agents on opposing teams can work together
- make some payoff matrix with [0, 1] for probability of reproducing

In [65]:
class Team(Enum):
    A = 0
    B = 1

class Agent():
    def __init__(self, team, opinions):
        self.team: Team = team
        self.opinions: List = opinions
        self.probReproduce = 0
        self.childOpinions = []


In [147]:
TOLERANCE = 0.75
# [[(A, A), (A, B)]
#  [(B, A), (B, B)]]
UTILITY_MATRIX = [[1.0, 1.5],
                 [1.5, 1.0]]
def getUtility(agent1: Agent, agent2: Agent):
    return UTILITY_MATRIX[agent1.team.value][agent2.team.value]

In [158]:
numAgents = 100 # always even

def generateAgents(distribution: str) -> tuple[List[Agent], List[Agent]]:
    teamA = []
    teamB = []
    # [mean, std_dev, low bound, upper bound]
    teamADistribution = [-0.5, 0.25, -1, 0]
    teamBDistribution = [0.5, 0.25, 0, 1]

    if distribution == "normal":
        teamAOpinions = np.clip(np.random.normal(teamADistribution[0], teamADistribution[1], numAgents//2), teamADistribution[2], teamADistribution[3])
        teamBOpinions = np.clip(np.random.normal(teamBDistribution[0], teamBDistribution[1], numAgents//2), teamBDistribution[2], teamBDistribution[3])
    elif distribution == "uniform":
        teamAOpinions = np.clip(np.random.uniform(teamADistribution[2], teamADistribution[3], numAgents//2), teamADistribution[2], teamADistribution[3])
        teamBOpinions = np.clip(np.random.uniform(teamBDistribution[2], teamBDistribution[3], numAgents//2), teamBDistribution[2], teamBDistribution[3])

    print(teamAOpinions)
    print(teamBOpinions)
    for i in range(0, numAgents):
        if i < numAgents/2:
            # team A
            team = Team.A
            opinion = teamAOpinions[i]
            teamA.append(Agent(team, [opinion]))
        else:
            # team B
            team = Team.B
            opinion = teamBOpinions[i - numAgents//2]
            teamB.append(Agent(team, [opinion]))

    return teamA, teamB
        

In [159]:
teamA, teamB = generateAgents("normal")

[-0.29615237 -0.45592812 -0.28784991 -0.68771221 -0.64103427 -0.4427407
 -0.83954405 -0.00664676 -0.04457089 -0.4399996  -0.450984   -0.06297448
 -0.40184976 -0.0950766  -0.8749068  -0.5550351  -0.57201196 -0.56909548
 -0.58504014 -0.73967413 -0.77861408 -0.91964215 -0.540421   -0.2752368
 -0.52879415 -0.32271249 -0.77597025 -0.24879443 -0.18727124 -0.67042802
 -0.32776568 -0.76768347 -0.36369787 -0.22259601 -0.69389702 -0.43765886
 -0.63465774 -0.56841469 -0.43787381 -0.56263379 -0.47792829 -0.43932466
 -0.22767318 -0.64086885 -0.19661705 -0.78256181 -0.77842401 -0.5079842
 -0.3765967  -0.35957003]
[0.43230107 0.         0.11870323 0.52567904 0.45131248 0.6898967
 0.36976828 0.53734319 0.81494412 0.51723694 0.04382947 0.71475705
 0.42116131 0.61469559 0.84678562 0.50598114 0.36697238 0.45671995
 0.59578457 0.31509751 0.57664724 0.09955044 0.65086416 0.37055002
 0.30587732 0.4262326  0.5218278  0.38279414 0.30572579 1.
 0.33625687 0.2940022  0.52941093 0.55041966 0.36717834 0.45505017


In [160]:
print(teamA[0].opinions[0])

-0.2961523702205924


In [161]:
# teamA, teamB = generateAgents("uniform")

In [162]:
def updateTeam(agents: List[Agent]):
    newTeam = []
    print(agents)
    team = agents[0].team
    for agent in agents:
        numChildren = int(agent.probReproduce)
        if random.random() < (agent.probReproduce - numChildren):
            numChildren += 1

        # print(numChildren)
        
        for i in range(0, numChildren):
            # make new agents
            newTeam.append(Agent(team, agent.childOpinions))

    return newTeam


In [185]:
FRACTION_MOVE = 0.25
def sim(teamA: List[Agent], teamB: List[Agent], numIterations: int):
    teamAUpdate = teamA
    teamBUpdate = teamB

    for i in range(0, numIterations):
        # combine both lists
        combinedAgents = teamAUpdate + teamBUpdate
        # shuffle the combined list
        random.shuffle(combinedAgents)
        # take pairs of agents from the combined list
        agentPairs = []
        while len(combinedAgents) >= 2: # if odd number the last agents just dies (maybe some other way to solve this)
            agentPairs.append((combinedAgents.pop(), combinedAgents.pop()))
        
        # check if agents can work together pair by pair - either same team auto yes, diff team check threshold, or both cases check threshold
        for pair in agentPairs:
            delta = abs(pair[0].opinions[0] - pair[1].opinions[0])
            print(f"same team: {pair[0].team == pair[1].team}, delta: {delta}")
            if delta <= TOLERANCE:
                # can work together
                utility = getUtility(pair[0], pair[1])
                # child opinions will move closer the midpoint between agents by the delta/4
                if pair[0].team != pair[1].team:
                    # diff teams, put a cap at 0
                    if pair[0].opinions[0] < pair[1].opinions[0]:
                        agent0ChildOpinions = min(0, pair[0].opinions[0] + delta*FRACTION_MOVE)
                        agent1ChildOpinions = max(0, pair[1].opinions[0] - delta*FRACTION_MOVE)
                    else:
                        agent0ChildOpinions = max(0, pair[0].opinions[0] - delta*FRACTION_MOVE)
                        agent1ChildOpinions = min(0, pair[1].opinions[0] + delta*FRACTION_MOVE)
                else:                
                    if pair[0].opinions[0] < pair[1].opinions[0]:
                        agent0ChildOpinions = pair[0].opinions[0] + delta*FRACTION_MOVE
                        agent1ChildOpinions = pair[0].opinions[0] - delta*FRACTION_MOVE
                    else:
                        agent0ChildOpinions = pair[0].opinions[0] - delta*FRACTION_MOVE
                        agent1ChildOpinions = pair[0].opinions[0] + delta*FRACTION_MOVE
            else:
                # can't work together
                utility = 0
                agent0ChildOpinions = []
                agent1ChildOpinions = []

            pair[0].probReproduce = utility
            pair[0].childOpinions = [agent0ChildOpinions]
            pair[1].probReproduce = utility
            pair[1].childOpinions = [agent1ChildOpinions]

            print(f"utility: {pair[0].probReproduce}")

        teamAUpdate = updateTeam(teamAUpdate)
        teamBUpdate = updateTeam(teamBUpdate)
                
        # if they can't then prob reproduce is 0
        # if they can then get prob reporduce from utility
        # calculate opinions of offspring
    return teamAUpdate, teamBUpdate

In [190]:
teamANew, teamBNew = sim(teamA, teamB, 10)

same team: False, delta: 0.6020296882373287
utility: 1.5
same team: False, delta: 1.0921003476883182
utility: 0
same team: True, delta: 0.1314366265607212
utility: 1.0
same team: False, delta: 0.9650037054689755
utility: 0
same team: True, delta: 0.28586244172840597
utility: 1.0
same team: False, delta: 0.8451066333483003
utility: 0
same team: True, delta: 0.2133364671650403
utility: 1.0
same team: False, delta: 1.0955800509654043
utility: 0
same team: True, delta: 0.43230106534172397
utility: 1.0
same team: False, delta: 0.8912919679150622
utility: 0
same team: True, delta: 0.08450015408402511
utility: 1.0
same team: True, delta: 0.5600721180570951
utility: 1.0
same team: False, delta: 0.9936100277816111
utility: 0
same team: True, delta: 0.5457922498373056
utility: 1.0
same team: True, delta: 0.2096748561434294
utility: 1.0
same team: False, delta: 0.7489450892050371
utility: 1.5
same team: True, delta: 0.43779274705940097
utility: 1.0
same team: False, delta: 1.2233079489427694
util

In [191]:
print(f"teamASize: {len(teamANew)}, teamBSize: {len(teamBNew)}")

teamASize: 165, teamBSize: 157
